In [4]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

# Define sentiment labels for the model
labels = ["Negative", "Neutral", "Positive"]

# Step 1: Load Dataset (replace with your dataset path)
# Assume dataset has two columns: 'text' and 'label'
dataset_path = "train_text.csv"  # Replace with your dataset file
data = pd.read_csv(dataset_path)

# Convert numeric labels (if any) to string (e.g., 0 -> Negative, 1 -> Neutral, 2 -> Positive)
label_mapping = {0: "Negative", 1: "Neutral", 2: "Positive"}
data["true_label"] = data["true_label"].map(label_mapping)

# Step 2: Initialize ground truth and predictions lists
y_true = []  # Ground truth labels
y_pred = []  # Model predictions

# Step 3: Make predictions using the model
for _, row in data.iterrows():
    tweet = row["text"]
    true_label = row["true_label"]
    
    # Tokenize the tweet
    encoded_tweet = tokenizer(tweet, return_tensors="pt", truncation=True)
    
    # Predict
    output = model(**encoded_tweet)
    scores = output.logits.detach().numpy()
    probs = softmax(scores, axis=1)  # Convert logits to probabilities
    
    # Get the highest probability label
    predicted_label = labels[np.argmax(probs)]
    
    # Append to lists
    y_true.append(true_label)
    y_pred.append(predicted_label)

# Step 4: Evaluate the Model
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=labels))

print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_true, y_pred, labels=labels)
print(conf_matrix)


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Classification Report:
              precision    recall  f1-score   support

    Negative       0.69      0.86      0.77        21
     Neutral       0.75      0.79      0.77        70
    Positive       0.80      0.68      0.73        53

    accuracy                           0.76       144
   macro avg       0.75      0.77      0.76       144
weighted avg       0.76      0.76      0.76       144


Confusion Matrix:
[[18  1  2]
 [ 8 55  7]
 [ 0 17 36]]
